In [1]:
import asyncio
import logging
from lockstar_general.backend.BackendResponse import BackendResponse
from lockstar_general.backend.BackendCall import BackendCall
from lockstar_client.BufferBaseClient_ import BufferBaseClient_
import numpy as np


In [2]:
class FGClient(BufferBaseClient_):
    """Basic Module which implements a simple PID controller by using input_1 as error signal,
    input_2 as setpoint and output 1 for the control signal"""

    def __init__(self, lockstar_ip, lockstar_port, client_id) -> None:
        super().__init__(lockstar_ip, lockstar_port, client_id, "FGModule")

    async def set_cfunction(self, func: str, scale: str):
        bc = BackendCall(
            self.client_id,
            "FGModule",
            "set_cfunction",
            args={"func": func, "scale": scale},
        )
        return await self._call_lockstar(bc)

    async def start_ccalculation(
        self
    ):
        bc = BackendCall(
            self.client_id,
            "FGModule",
            "start_ccalculation",
            args={},
        )
        return await self._call_lockstar(bc)

    async def start_output(self):
        bc = BackendCall(self.client_id, "FGModule", "start_output", args={})
        return await self._call_lockstar(bc)


In [3]:
client = FGClient('192.168.137.2', 10780, 1234)
await client.register_client_id()

True

In [4]:
async def prep_output(func_buffer):
    sampling_rate = 300000
    await client.set_sampling_rate(sampling_rate)
    await client.set_ch_one_output_limits(-10, 10)
    await client.set_ch_two_output_limits(-10, 10)
    await client.unclamp_output()
    await client.set_ch_func_buffer(func_buffer, buffer_one=True)
    await client.set_ch_func_buffer(func_buffer, buffer_one=False)
    await client.start_ccalculation()

In [57]:
func_buffer = [
    {
        "ll_func": 0x00000001,
        "ll_scaling": 0x00000000,
        "start_value": 0,
        "step_size": 2147483,
        "num_samples": 2000,
        "total_scaling": 4,
        "offset": 0,
        "n_periods": 100,
        "time_start": 2,
    },
    {
    "ll_func": 0x00000004,
    "ll_scaling": 0x00000600,
    "start_value": -214748364,
    "step_size": 185899,
    "num_samples": 2000,
    "total_scaling": -568.0519530539988,
    "offset": 4,
    "n_periods": 10,
    "time_start": 2,
    },
    {
    "ll_func": 0x00000001,
    "ll_scaling": 0x00000000,
    "start_value": 0,
    "step_size": 429497,
    "num_samples": 10000,
    "total_scaling": 6,
    "offset": 0,
    "n_periods": 20,
    "time_start": 2,
    },
]

In [58]:
await prep_output(func_buffer)

In [59]:
await client.start_output()

ACK

In [244]:
await client.set_ch_func_buffer(func_buffer, buffer_one=True)
await client.set_ch_func_buffer(func_buffer, buffer_one=False)

ACK